- **Binary Tree**: Tree structure of data where each node has two children.
  - **Complete Binary Tree** has all of its members filled.
- **Merkle Tree**: A complete binary tree where each node is the hash of the left child and the right child.

In [8]:
#Source: https://stackoverflow.com/users/19268430/alejandro-mera
class Node(object):
    def __init__(self, val, left=None, right=None ):
        self.val = val
        self.left = left
        self.right = right

    def __str__(self):
        return str(self.val)

def PrintTree(root, hashes=[] ):
    def height(root):
        return 1 + max(height(root.left), height(root.right)) if root else -1  
    nlevels = height(root)
    width =  pow(2,nlevels+1)

    q=[(root,0,width,'c')]
    levels=[]

    while(q):
        node,level,x,align= q.pop(0)
        if node:            
            if len(levels)<=level:
                levels.append([])
        
            levels[level].append([node,level,x,align])
            seg= width//(pow(2,level+1))
            q.append((node.left,level+1,x-seg,'l'))
            q.append((node.right,level+1,x+seg,'r'))

    for i,l in enumerate(levels):
        pre=0
        preline=0
        linestr=''
        pstr=''
        seg= width//(pow(2,i+1))
        for n in l:
            valstr= str(n[0]) if len(hashes) == 0 or n[0].val in hashes else "X";
            if n[3]=='r':
                linestr+=' '*(n[2]-preline-1-seg-seg//2)+ '¯'*(seg +seg//2)+'\\'
                preline = n[2] 
            if n[3]=='l':
               linestr+=' '*(n[2]-preline-1)+'/' + '¯'*(seg+seg//2)  
               preline = n[2] + seg + seg//2
            pstr+=' '*(n[2]-pre-len(valstr))+valstr #correct the potition acording to the number size
            pre = n[2]
        print(linestr)
        print(pstr)   


In [4]:
import math

def _crc( data, len):
  crc = 0;
  for i in range(0,len):
    inbyte = data[i];
    for j in range(0,8):
      mix = (crc ^ inbyte) & 0x01;
      crc >>= 1;
      if ( mix ):
        crc ^= 0x8C;
      inbyte >>= 1;
  return crc;

def _hash( data ):
    return f'{( _crc( bytes(data,"ascii") , len(data) ) ):x}'
 
def summarize( leaves, nodes ):
    print("""
        Root:\t{}
        Depth:\t{}
        leaves:\t{}
        nodes:\t{}""".format(nodes.val,
                             int(math.log2(len(leaves))),
                             len(leaves),
                             2**(int(math.log2(len(leaves)))+1)-1 ))

def padData( data ):
    v = math.log2(len(leaves))
    if ( not v.is_integer() ):
        size = 2**math.ceil( v );
        e2 = data[ -1 ];
        if len(data) %2 == 0:
            e1 = data[ -2 ];
            while len( data ) < size:
                data.append( e1 );
                data.append( e2 );
        else:
            while len( data ) < size:
                data.append( e2 );
    return data;
    
def makeTree( leaves ):
    nodes = [ Node( _hash( str( i ) ) ) for i in padData(leaves) ];

    while len(nodes) > 1:
        arr = [];
        for i in range( 0 , len( nodes ), 2 ):
            arr.append( Node( _hash( "{}{}".format( nodes[i].val , nodes[i+1].val ) ) , nodes[i] , nodes[i+1] ) );
        nodes = arr;
    return nodes[0];

def treeToArray( root ): #= nodes[0];
    depth = int(math.log2(len(leaves)));
    arr = [];
    while depth > 0:
        flow = 0;
        leafCount = 2**depth;
        while flow < leafCount:
            node = root;
            i = 0;
            while i < depth:
                #The route to each leaf is equivalent to counting in binary
                #e.g LLL = 0b000, LLR = 0b001, LRL = 0b010 ...
                if ( flow >> ( depth - i -1 )  & 1 == 1 ):
                    node = node.right;
                else:
                    node = node.left;
                i +=1;
            arr.append( node.val );
            flow += 1;
        depth -= 1;
    return arr;

In [18]:
leaves = [ i for i in range(0,12) ];
print(leaves)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [19]:
tree = makeTree( leaves );

PrintTree( tree )
summarize( leaves , tree );


                              14
               /¯¯¯¯¯¯¯¯¯¯¯¯       ¯¯¯¯¯¯¯¯¯¯¯¯\
               6                              36
       /¯¯¯¯¯¯   ¯¯¯¯¯¯\               /¯¯¯¯¯¯   ¯¯¯¯¯¯\
      56              3f              d5              3a
   /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\
  1c      c4      51       7      31      8a      8a      8a
 /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\
be  e0   2  5c  df  81  63  3d  7c  22  57   9  57   9  57   9

        Root:	14
        Depth:	4
        leaves:	16
        nodes:	31


In [12]:
print( treeToArray( tree ) )

['be', 'e0', '2', '5c', 'df', '81', '63', '3d', '7c', '22', '7c', '22', '7c', '22', '7c', '22', '1c', 'c4', '51', '7', '31', '31', '31', '31', '56', '3f', '2d', '2d', '6', 'cc']


In [13]:
treeArray = treeToArray( tree );

aHash = "5c";
index = treeArray.index( aHash );
depth = int( math.log2(len( treeArray ) ) );

print("""Flattened tree: {}

To prove leaf hash [{}] at index {} is in the tree, you need:""".format(treeArray,aHash,index));

hashArray = [];
offset = 0;
while depth > 0:
    #if index is even we need the right hash, else left,
    if ( index) % 2 == 0:
        hashArray.append(treeArray[offset + index + 1]);
        print("\thash at index: {}".format(offset + index + 1));
    else:
        hashArray.append(treeArray[offset + index - 1]);
        print("\thash at index: {}".format(offset + index - 1));
    #move on to the new layer of the tree, and update index = ( index / 2 );
    offset += ( 2**(depth) );
    index = int(index/2.0);
    depth -=1;
    
PrintTree( tree, hashArray )

Flattened tree: ['be', 'e0', '2', '5c', 'df', '81', '63', '3d', '7c', '22', '7c', '22', '7c', '22', '7c', '22', '1c', 'c4', '51', '7', '31', '31', '31', '31', '56', '3f', '2d', '2d', '6', 'cc']

To prove leaf hash [5c] at index 3 is in the tree, you need:
	hash at index: 2
	hash at index: 16
	hash at index: 25
	hash at index: 29

                               X
               /¯¯¯¯¯¯¯¯¯¯¯¯       ¯¯¯¯¯¯¯¯¯¯¯¯\
               X                              cc
       /¯¯¯¯¯¯   ¯¯¯¯¯¯\               /¯¯¯¯¯¯   ¯¯¯¯¯¯\
       X              3f               X               X
   /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\       /¯¯¯ ¯¯¯\
  1c       X       X       X       X       X       X       X
 /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\   /¯ ¯\
 X   X   2   X   X   X   X   X   X   X   X   X   X   X   X   X


In [14]:
def proove( leaf, anIndex, hashes):
    aHash = leaf;
    for i in range( 0, len(hashes) ):
        tmp = aHash
        if anIndex%2 == 1:
            aHash = _hash("{}{}".format(hashes[i],tmp))
        else:
            aHash = _hash("{}{}".format(tmp,hashes[i]))
        anIndex = int( anIndex / 2.0 );
    return aHash;

In [15]:
rslt = proove( aHash, treeArray.index(aHash) , hashArray )

print("""
    tree root: {}
    proof rslt: {}
    leaf: {}
    is in tree: {}""".format( tree.val, rslt , aHash, rslt == tree.val))


    tree root: 87
    proof rslt: 87
    leaf: 5c
    is in tree: True
